In [89]:
import pandas as pd
import numpy as np
import ast
import regex as re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt

In [90]:
# Set the display options to show more rows and columns
pd.set_option('display.max_rows', None)  # Show all rows
pd.set_option('display.max_columns', None)  # Show all columns

In [91]:
df = pd.read_csv('./data/apartments.csv')
df.head(2)

,PropertyName,PropertySubName,NearbyLocations,LocationAdvantages,Link,PriceDetails,TopFacilities
0,Smartworld One DXP,"2, 3, 4 BHK Apartment in Sector 113, Gurgaon","['Bajghera Road', 'Palam Vihar Halt', 'DPSG Pa...","{'Bajghera Road': '800 Meter', 'Palam Vihar Ha...",https://www.99acres.com/smartworld-one-dxp-sec...,"{'2 BHK': {'building_type': 'Apartment', 'area...","['Swimming Pool', 'Salon', 'Restaurant', 'Spa'..."
1,M3M Crown,"3, 4 BHK Apartment in Sector 111, Gurgaon","['DPSG Palam Vihar Gurugram', 'The NorthCap Un...","{'DPSG Palam Vihar Gurugram': '1.4 Km', 'The N...",https://www.99acres.com/m3m-crown-sector-111-g...,"{'3 BHK': {'building_type': 'Apartment', 'area...","['Bowling Alley', 'Mini Theatre', 'Manicured G..."


In [92]:
df.drop(index=22, axis=0, inplace=True)
df.columns

Index(['PropertyName', 'PropertySubName', 'NearbyLocations',
       'LocationAdvantages', 'Link', 'PriceDetails', 'TopFacilities'],
      dtype='object')

In [93]:
samp = df.sample()
print(samp['NearbyLocations'])
samp['LocationAdvantages']

103    ['Old Delhi Gurgaon Road', 'Hyatt Place Gurgao...
Name: NearbyLocations, dtype: object


103    {'Old Delhi Gurgaon Road': '61 Meter', 'Hyatt ...
Name: LocationAdvantages, dtype: object

In [94]:
df['PriceDetails'][23]

"{'2 BHK': {'building_type': 'Apartment', 'area_type': 'Carpet Area', 'area': '1,300 - 1,330 sq.ft.', 'price-range': '₹ 1.3 - 1.56 Cr'}, '3 BHK': {'building_type': 'Apartment', 'area_type': 'Carpet Area', 'area': '1,650 - 1,700 sq.ft.', 'price-range': '₹ 1.65 - 2.06 Cr'}, '4 BHK': {'building_type': 'Apartment', 'area_type': 'Carpet Area', 'area': '2,055 - 2,249 sq.ft.', 'price-range': '₹ 3.29 - 3.69 Cr'}}"

In [95]:
df['TopFacilities'][0]

"['Swimming Pool', 'Salon', 'Restaurant', 'Spa', 'Cafeteria', 'Sun Deck', '24x7 Security', 'Club House', 'Gated Community']"

# Create 3 recomender systems.
A. 'TopFacilities' Based
B. 'PriceDetails' Based
C. 'LocationAdvantages' Based

# A. 'TopFacilities' :
Convert word into vectors and find 5 nearest vectors to it in the same space

In [96]:
df[['PropertyName', 'TopFacilities']]['TopFacilities'][0]

"['Swimming Pool', 'Salon', 'Restaurant', 'Spa', 'Cafeteria', 'Sun Deck', '24x7 Security', 'Club House', 'Gated Community']"

In [97]:
# def extract_list(s):
#     return re.findall(r"'(.*?')",s)

# df['TopFacilities_NS'] = df[['PropertyName', 'TopFacilities']]['TopFacilities'].apply(extract_list)

In [98]:
# remove all characters except Alphaumeric and space from the string
# this will convert this text into desired format 
def rep_extra(st):
    return re.sub(r'[^a-zA-Z0-9\s]', '', st)

df['TopFacilitiesStr'] = df['TopFacilities'].apply(rep_extra)


In [99]:
tfidf_vectorizer = TfidfVectorizer(stop_words='english', ngram_range=(1,2))

In [100]:
tfidf_matrix = tfidf_vectorizer.fit_transform(df['TopFacilitiesStr'])
tfidf_matrix

<246x966 sparse matrix of type '<class 'numpy.float64'>'
	with 7503 stored elements in Compressed Sparse Row format>

In [101]:
tfidf_matrix.toarray().shape

(246, 966)

In [102]:
tfidf_matrix.toarray()[0]

array([0.        , 0.        , 0.        , 0.18809342, 0.18809342,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

This being high dimensionality data, we will use cosine similarity rather than Eucledian distance measure

In [103]:
cosine_sim1 = cosine_similarity(tfidf_matrix, tfidf_matrix)
cosine_sim1.shape

(246, 246)

In [104]:
# idx = df[df['PropertyName'] == "Adani Brahma Samsara Vilasa"].index[0]
# sim_scores = cosine_sim1[idx]
# sim_scores.shape
# sim_scr = pd.Series(sim_scores, index=df.index)

# sim_scr = sim_scr.sort_values(ascending=False)

# similar_indexes = sim_scr.iloc[1:6].index

# #df['PropertyName'].iloc[similar_indexes.to_list()]
# recom_df = pd.DataFrame(index= similar_indexes, data={
#         'PropertyName': df['PropertyName'].iloc[similar_indexes.to_list()],
#         'SimilarityScore': sim_scr.iloc[1:6]
#     })
# recom_df
# df['PropertyName'].iloc[similar_indexes.to_list()]

In [105]:
def recommend_properties(property_name, cosine_sim=cosine_sim1):
    # Get the index of the property that matches the name
    idx = df.index[df['PropertyName'] == property_name].tolist()[0]

    # Get the pairwise similarity scores with that property
    sim_scores = list(enumerate(cosine_sim1[idx]))

    # Sort the properties based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 5 most similar properties
    sim_scores = sim_scores[1:6]

    # Get the property indices
    property_indices = [i[0] for i in sim_scores]
    
    recommendations_df = pd.DataFrame({
        'PropertyName': df['PropertyName'].iloc[property_indices],
        'SimilarityScore': sim_scores
    })

    # Return the top 10 most similar properties
    return recommendations_df

recommend_properties("Adani Brahma Samsara Vilasa", cosine_sim=cosine_sim1)

,PropertyName,SimilarityScore
153,JMS Prime Land,"(152, 0.30919160259213807)"
133,ROF Insignia Park 2,"(132, 0.30847156669739634)"
54,Birla Navya Avik,"(53, 0.29853517564596366)"
152,Ashiana Amarah,"(151, 0.2759030460276673)"
16,M3M Golf Hills,"(16, 0.2659184992552591)"


In [106]:
recommend_properties("DLF The Arbour")

,PropertyName,SimilarityScore
64,Ace Palm Floors,"(63, 0.530580608798138)"
217,Yashika 104,"(216, 0.46857836608919934)"
18,Whiteland Blissville,"(18, 0.4525174081848686)"
93,JMS The Nation,"(92, 0.45142781760536815)"
63,Vatika Aspiration,"(62, 0.4251977706031502)"


# B. Recommender system on 'PriceDetails'
1. Format the text to create columns
2. OneHot encode categorical columns
3. Standardize all columns
4. Create cosine similarities
5. Select top recommendations based on cosine similarities

In [107]:
df[['PropertyName', 'PriceDetails']]['PriceDetails'].sample(1)

228    {'2 BHK': {'building_type': 'Apartment', 'area...
Name: PriceDetails, dtype: object

In [108]:
import pandas as pd
import json

# Load the dataset
df_appartments = pd.read_csv('./data/apartments.csv').drop(22)

# Function to parse and extract the required features from the PriceDetails column
def refined_parse_modified_v2(detail_str):
    try:
        details = json.loads(detail_str.replace("'", "\""))
    except:
        return {}

    extracted = {}
    for bhk, detail in details.items():
        # Extract building type
        extracted[f'building type_{bhk}'] = detail.get('building_type')

        # Parsing area details
        area = detail.get('area', '')
        area_parts = area.split('-')
        if len(area_parts) == 1:
            try:
                value = float(area_parts[0].replace(',', '').replace(' sq.ft.', '').strip())
                extracted[f'area low {bhk}'] = value
                extracted[f'area high {bhk}'] = value
            except:
                extracted[f'area low {bhk}'] = None
                extracted[f'area high {bhk}'] = None
        elif len(area_parts) == 2:
            try:
                extracted[f'area low {bhk}'] = float(area_parts[0].replace(',', '').replace(' sq.ft.', '').strip())
                extracted[f'area high {bhk}'] = float(area_parts[1].replace(',', '').replace(' sq.ft.', '').strip())
            except:
                extracted[f'area low {bhk}'] = None
                extracted[f'area high {bhk}'] = None

        # Parsing price details
        price_range = detail.get('price-range', '')
        price_parts = price_range.split('-')
        if len(price_parts) == 2:
            try:
                extracted[f'price low {bhk}'] = float(price_parts[0].replace('₹', '').replace(' Cr', '').replace(' L', '').strip())
                extracted[f'price high {bhk}'] = float(price_parts[1].replace('₹', '').replace(' Cr', '').replace(' L', '').strip())
                if 'L' in price_parts[0]:
                    extracted[f'price low {bhk}'] /= 100
                if 'L' in price_parts[1]:
                    extracted[f'price high {bhk}'] /= 100
            except:
                extracted[f'price low {bhk}'] = None
                extracted[f'price high {bhk}'] = None

    return extracted
# Apply the refined parsing and generate the new DataFrame structure
data_refined = []

for _, row in df_appartments.iterrows():
    features = refined_parse_modified_v2(row['PriceDetails'])
    
    # Construct a new row for the transformed dataframe
    new_row = {'PropertyName': row['PropertyName']}
    
    # Populate the new row with extracted features
    for config in ['1 BHK', '2 BHK', '3 BHK', '4 BHK', '5 BHK', '6 BHK', '1 RK', 'Land']:
        new_row[f'building type_{config}'] = features.get(f'building type_{config}')
        new_row[f'area low {config}'] = features.get(f'area low {config}')
        new_row[f'area high {config}'] = features.get(f'area high {config}')
        new_row[f'price low {config}'] = features.get(f'price low {config}')
        new_row[f'price high {config}'] = features.get(f'price high {config}')
    
    data_refined.append(new_row)

df_final_refined_v2 = pd.DataFrame(data_refined).set_index('PropertyName')

In [109]:
df_final_refined_v2['building type_Land'] = df_final_refined_v2['building type_Land'].replace({'':'Land'})

In [110]:
df['PriceDetails'][10]

"{'2 BHK': {'building_type': 'Independent Floor', 'area_type': 'Carpet Area', 'area': '1,055 sq.ft.', 'price-range': '₹ 1.05 - 1.5 Cr'}, '3 BHK': {'building_type': 'Independent Floor', 'area_type': 'Carpet Area', 'area': '1,325 - 1,525 sq.ft.', 'price-range': '₹ 1.35 - 1.84 Cr'}}"

In [111]:
df_final_refined_v2.head(2)

,building type_1 BHK,area low 1 BHK,area high 1 BHK,price low 1 BHK,price high 1 BHK,building type_2 BHK,area low 2 BHK,area high 2 BHK,price low 2 BHK,price high 2 BHK,building type_3 BHK,area low 3 BHK,area high 3 BHK,price low 3 BHK,price high 3 BHK,building type_4 BHK,area low 4 BHK,area high 4 BHK,price low 4 BHK,price high 4 BHK,building type_5 BHK,area low 5 BHK,area high 5 BHK,price low 5 BHK,price high 5 BHK,building type_6 BHK,area low 6 BHK,area high 6 BHK,price low 6 BHK,price high 6 BHK,building type_1 RK,area low 1 RK,area high 1 RK,price low 1 RK,price high 1 RK,building type_Land,area low Land,area high Land,price low Land,price high Land
PropertyName,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Smartworld One DXP,None,NaN,NaN,NaN,NaN,Apartment,1370.0,1370.0,2.0,2.4,Apartment,1850.0,2050.0,2.25,3.59,Apartment,2600.0,2600.0,3.24,4.56,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN
M3M Crown,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,Apartment,1605.0,2170.0,2.20,3.03,Apartment,2248.0,2670.0,3.08,3.73,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN


In [112]:
categorical_columns = df_final_refined_v2.select_dtypes(include=['object']).columns.tolist()
categorical_columns

['building type_1 BHK',
 'building type_2 BHK',
 'building type_3 BHK',
 'building type_4 BHK',
 'building type_5 BHK',
 'building type_6 BHK',
 'building type_1 RK',
 'building type_Land']

In [113]:
ohe_df = pd.get_dummies(df_final_refined_v2, columns=categorical_columns, drop_first=True)
ohe_df.fillna(0,inplace=True)
ohe_df.head(2)

,area low 1 BHK,area high 1 BHK,price low 1 BHK,price high 1 BHK,area low 2 BHK,area high 2 BHK,price low 2 BHK,price high 2 BHK,area low 3 BHK,area high 3 BHK,price low 3 BHK,price high 3 BHK,area low 4 BHK,area high 4 BHK,price low 4 BHK,price high 4 BHK,area low 5 BHK,area high 5 BHK,price low 5 BHK,price high 5 BHK,area low 6 BHK,area high 6 BHK,price low 6 BHK,price high 6 BHK,area low 1 RK,area high 1 RK,price low 1 RK,price high 1 RK,area low Land,area high Land,price low Land,price high Land,building type_1 BHK_Service Apartment,building type_2 BHK_Independent Floor,building type_2 BHK_Service Apartment,building type_3 BHK_Independent Floor,building type_3 BHK_Service Apartment,building type_3 BHK_Villa,building type_4 BHK_Independent Floor,building type_4 BHK_Villa,building type_5 BHK_Independent Floor,building type_5 BHK_Villa,building type_6 BHK_Villa
PropertyName,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Smartworld One DXP,0.0,0.0,0.0,0.0,1370.0,1370.0,2.0,2.4,1850.0,2050.0,2.25,3.59,2600.0,2600.0,3.24,4.56,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0
M3M Crown,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1605.0,2170.0,2.20,3.03,2248.0,2670.0,3.08,3.73,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0


In [114]:
ohe_df.shape

(246, 43)

In [115]:
from sklearn.preprocessing import StandardScaler

# Initialize the scaler
scaler = StandardScaler()

# Apply the scaler to the entire dataframe
ohe_df_normalized = pd.DataFrame(scaler.fit_transform(ohe_df), columns=ohe_df.columns, index=ohe_df.index)
ohe_df_normalized.head()

,area low 1 BHK,area high 1 BHK,price low 1 BHK,price high 1 BHK,area low 2 BHK,area high 2 BHK,price low 2 BHK,price high 2 BHK,area low 3 BHK,area high 3 BHK,price low 3 BHK,price high 3 BHK,area low 4 BHK,area high 4 BHK,price low 4 BHK,price high 4 BHK,area low 5 BHK,area high 5 BHK,price low 5 BHK,price high 5 BHK,area low 6 BHK,area high 6 BHK,price low 6 BHK,price high 6 BHK,area low 1 RK,area high 1 RK,price low 1 RK,price high 1 RK,area low Land,area high Land,price low Land,price high Land,building type_1 BHK_Service Apartment,building type_2 BHK_Independent Floor,building type_2 BHK_Service Apartment,building type_3 BHK_Independent Floor,building type_3 BHK_Service Apartment,building type_3 BHK_Villa,building type_4 BHK_Independent Floor,building type_4 BHK_Villa,building type_5 BHK_Independent Floor,building type_5 BHK_Villa,building type_6 BHK_Villa
PropertyName,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Smartworld One DXP,-0.252266,-0.169584,-0.105197,-0.082332,1.223499,1.020101,-0.173712,1.158423,0.553787,0.370864,0.807098,0.515061,0.602838,0.212073,0.383381,0.242019,-0.468954,-0.460463,-0.248049,-0.235915,-0.125582,-0.118934,-0.077649,-0.073387,-0.105157,-0.10253,-0.090521,-0.082725,-0.447044,-0.371421,-0.195703,-0.240489,-0.111111,-0.289310,-0.063888,-0.372678,-0.063888,-0.171139,-0.254824,-0.236208,-0.111111,-0.216353,-0.063888
M3M Crown,-0.252266,-0.169584,-0.105197,-0.082332,-0.893541,-0.896660,-0.283546,-0.387986,0.293086,0.472749,0.772095,0.342903,0.382746,0.243172,0.341443,0.108677,-0.468954,-0.460463,-0.248049,-0.235915,-0.125582,-0.118934,-0.077649,-0.073387,-0.105157,-0.10253,-0.090521,-0.082725,-0.447044,-0.371421,-0.195703,-0.240489,-0.111111,-0.289310,-0.063888,-0.372678,-0.063888,-0.171139,-0.254824,-0.236208,-0.111111,-0.216353,-0.063888
Adani Brahma Samsara Vilasa,-0.252266,-0.169584,-0.105197,-0.082332,-0.893541,-0.896660,-0.283546,-0.387986,0.500583,1.304803,0.933110,4.253341,0.696627,1.056178,0.414835,3.124152,-0.468954,-0.460463,-0.248049,-0.235915,-0.125582,-0.118934,-0.077649,-0.073387,-0.105157,-0.10253,-0.090521,-0.082725,0.369534,1.828502,0.643875,7.635886,-0.111111,-0.289310,-0.063888,2.683282,-0.063888,-0.171139,3.924283,-0.236208,-0.111111,-0.216353,-0.063888
Sobha City,-0.252266,-0.169584,-0.105197,-0.082332,1.240497,1.470610,-0.198425,1.680336,0.405879,0.619632,0.464065,0.883970,0.492166,0.373342,0.189416,0.483000,-0.468954,-0.460463,-0.248049,-0.235915,-0.125582,-0.118934,-0.077649,-0.073387,-0.105157,-0.10253,-0.090521,-0.082725,-0.447044,-0.371421,-0.195703,-0.240489,-0.111111,-0.289310,-0.063888,-0.372678,-0.063888,-0.171139,-0.254824,-0.236208,-0.111111,-0.216353,-0.063888
Signature Global City 93,-0.252266,-0.169584,-0.105197,-0.082332,0.622383,0.667529,-0.232468,0.295011,-0.100626,-0.070634,0.016022,-0.142827,-1.022842,-0.943016,-0.465873,-0.490563,-0.468954,-0.460463,-0.248049,-0.235915,-0.125582,-0.118934,-0.077649,-0.073387,-0.105157,-0.10253,-0.090521,-0.082725,-0.447044,-0.371421,-0.195703,-0.240489,-0.111111,3.456497,-0.063888,2.683282,-0.063888,-0.171139,-0.254824,-0.236208,-0.111111,-0.216353,-0.063888


In [116]:
from sklearn.metrics.pairwise import cosine_similarity

# Compute the cosine similarity matrix
cosine_sim2 = cosine_similarity(ohe_df_normalized)
cosine_sim2.shape

(246, 246)

In [117]:
def recommend_properties_with_scores(property_name, top_n=247):
    
    # Get the similarity scores for the property using its name as the index
    sim_scores = list(enumerate(cosine_sim2[ohe_df_normalized.index.get_loc(property_name)]))
    
    # Sort properties based on the similarity scores
    sorted_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # Get the indices and scores of the top_n most similar properties
    top_indices = [i[0] for i in sorted_scores[1:top_n+1]]
    top_scores = [i[1] for i in sorted_scores[1:top_n+1]]
    
    # Retrieve the names of the top properties using the indices
    top_properties = ohe_df_normalized.index[top_indices].tolist()
    
    # Create a dataframe with the results
    recommendations_df = pd.DataFrame({
        'PropertyName': top_properties,
        'SimilarityScore': top_scores
    })
    
    return recommendations_df

# Test the recommender function using a property name
recommend_properties_with_scores('M3M Golf Hills').head(5)

,PropertyName,SimilarityScore
0,AIPL The Peaceful Homes,0.955462
1,Smartworld One DXP,0.954670
2,Unitech Escape,0.953092
3,M3M Capital,0.951156
4,BPTP Terra,0.943128


# C. LocationAdvantages Based Recommendar System

In [118]:
df[['PropertyName', 'LocationAdvantages']]['LocationAdvantages'][10]

"{'Naurangpur Cricket Stadium': '850 Meter', 'Naurangpur Road': '1.4 KM', 'National Highway  48': '5.7 KM', 'Vatika Town Square-INXT': '6.4 KM', 'Ompee International School': '7.9 KM', 'Manesar Bus Stand': '8.1 KM', 'Yashlok Medical Centre': '8.3 KM', 'Euro International School': '9 KM', 'Sushant University': '19.7 KM', 'Indira Gandhi Intl Airport': '32.1 KM'}"

In [119]:
def distance_to_meters(distance_str):
    try:
        if 'Km' in distance_str or 'KM' in distance_str:
            return float(distance_str.split()[0]) * 1000
        elif 'Meter' in distance_str or 'meter' in distance_str:
            return float(distance_str.split()[0])
        else:
            return None
    except:
        return None

In [120]:
# Extract distances for each location
location_matrix = {}
for index, row in df.iterrows():
    distances = {}
    for location, distance in ast.literal_eval(row['LocationAdvantages']).items():
        distances[location] = distance_to_meters(distance)
    location_matrix[index] = distances

# Convert the dictionary to a dataframe
location_df = pd.DataFrame.from_dict(location_matrix, orient='index')

# Display the first few rows
location_df.head()

,Bajghera Road,Palam Vihar Halt,DPSG Palam Vihar,Park Hospital,Gurgaon Railway Station,The NorthCap University,Dwarka Expy,Hyatt Place Gurgaon Udyog Vihar,"Dwarka Sector 21, Metro Station",Pacific D21 Mall,Indira Gandhi International Airport,Hamoni Golf Camp,Fun N Food Waterpark,Accenture DDC5,DPSG Palam Vihar Gurugram,"Park Hospital, Palam Vihar",Palam Vihar Halt Railway Station,Dwarka Sector 21 Metro Station,Dwarka Expressway,Fun N Food Water Park,Tau DeviLal Sports Complex,Hyatt Place,Altrade Business Centre,AIPL Business Club Sector 62,Heritage Xperiential Learning School,CK Birla Hospital,Paras Trinity Mall Sector 63,Rapid Metro Station Sector 56,De Adventure Park,Golf Course Ext Rd,DoubleTree by Hilton Hotel Gurgaon,KIIT College of Engineering Sohna Road,Mehrauli-Gurgaon Road,Nirvana Rd,TERI Golf Course,The Shikshiyan School,WTC Plaza,Luxus Haritma Resort,BSF Golf Course,Rions Hospital,Gurgaon,Dwarka Sector 21,Nehru Stadium,Fun N Food WaterPark,IGI Airport,Vasant Kunj,Pranavananda Int. School,DLF Site central office,Holiday Inn Gurugram Sector 90,Krishna Hospital,Royal Institute Of Science,Sapphire 83 Mall,NH48,Garhi Harsaru Junction,Manesar Golf Course,AapnoGhar,Vega Schools NH-8,DLF Corporate Greens,Miracles Apollo Cradle Hospital,Hyatt Regency Gurugram,NH 48,Golden Greens Golf & Resorts Limited,Mount Olympus Junior School,Miracles Apollo Hospital,NH -8,"Savoy Suites, Manesar",Golden Greens Golf & Resorts,IMT Manesar,Amity University Gurugram,Golf Course Extension Road,"Dwarka Expy, Sector 109","Euro International School, Sector- 109",Jai Sai Ram Hospital,Aryan Hospital,Idea Cosmic Plaza,Indira Gandhi Intl Airport,Royal Institute Of Science & Management,Pataudi Road,Holiday Inn Sector 90,RPS International School,Aarvy Healthcare Hospital,Iris Broadway Mall,Imperia Mindspace,AIPL Business Tower,Heritage School,"Lotus Valley Intl School, Gurgaon",Gurugram University,Sector 55-56 Metro Station,Omaxe Gurgaon Mall,Sushant University,"Badshahpur Sohna Rd Hwy,Sector 48",Naurangpur Cricket Stadium,Naurangpur Road,National Highway 48,Vatika Town Square-INXT,Ompee International School,Manesar Bus Stand,Yashlok Medical Centre,Euro International School,WorldMark Gurgaon,Capital Cyberscape,The Shriram Millennium School,DoubleTree by Hilton Hotel,Badshahpur Sohna Hwy,Nakhrola Stadium,Delhi - Jaipur Expressway,Vatika Town Square-INXT Mall,Savoy Suites,Bal Bharati Public School,Vatika Business Centre,Indira Gandhi Int. Airport,St. Xavier's High School,Miracles Apollo Cradle,Ambience Mall New,NH8,Hyatt Regency Gurgaon,Delhi Public School,Elan Miracle Mall,Miracles Apollo Cradle Spectra Hospital,Agri Business Management Collage,Delhi Jaipur Expressway,Grand Hyatt Gurgaon,Duke Horse Riding Club,PVR Drive In Cinema,W Pratiksha Hospital,Metro World Mall,Unicosmos School,Faridabad Gurgaon Road,Sohna Road,Bestech Business Tower,Appu Ghar,SkyJumper Trampoline Park,Axis Bank,KMP Expressway,Karma Lakelands,Jungle Safari & Trails,DPS Manesar,Medanta Hospital,Faridabad - Gurgaon Road,Lingaya's Lalita Devi Institute,ASF Insignia SEZ,Banjara Market Gurugram,Central Plaza Mall,"Paras Hospitals, Gurgaon",Badshahpur Sohna Rd Hwy,Vega School,Indian School of Hospitality,Vatika City Centre,Aatish Hospital,Info Technology Park Phase 2,Huda Metro Station,Southern Peripheral Rd,Global Ways School,Radisson Hotel,NH 248A,Sector 55/56 Metro Station,Mavens Inn,Sanar International Hospital,Sector 53-54 Metro Station,"IILM University, Gurugram",The Banyan Tree World School,The Big Tree Cafe,DLF Golf and Country Club,"Delhi Public School, Sector 84",Aarvy Hospital,DPG Degree College,Shivani public school,Baghera University,Kutumbh Hospital,Bijwasan Railway Station,Global Foyer Mall,Phase 2 Metro Station,Gurgaon Dreamz Mall,"Metro Hospital, Palam Vihar",Delhi Ajmer Expressway,Infinity Business Park,Huda metro station,Rion's Hospital,"Euro International School, Sector- 109.",Golf Course Ext Road,"Heritage Xperiential Learning, CRPF Rd",Sector 54 Chowk Metro Station

In [121]:
location_df.columns[10:50]

Index(['Indira Gandhi International Airport', 'Hamoni Golf Camp',
       'Fun N Food Waterpark', 'Accenture DDC5', 'DPSG Palam Vihar Gurugram',
       'Park Hospital, Palam Vihar', 'Palam Vihar Halt Railway Station',
       'Dwarka Sector 21 Metro Station', 'Dwarka Expressway',
       'Fun N Food Water Park', 'Tau DeviLal Sports Complex', 'Hyatt Place',
       'Altrade Business Centre', 'AIPL Business Club Sector 62',
       'Heritage Xperiential Learning School', 'CK Birla Hospital',
       'Paras Trinity Mall Sector 63', 'Rapid Metro Station Sector 56',
       'De Adventure Park', 'Golf Course Ext Rd',
       'DoubleTree by Hilton Hotel Gurgaon',
       'KIIT College of Engineering Sohna Road', 'Mehrauli-Gurgaon Road',
       'Nirvana Rd', 'TERI Golf Course', 'The Shikshiyan School', 'WTC Plaza',
       'Luxus Haritma Resort', 'BSF Golf Course', 'Rions Hospital', 'Gurgaon',
       'Dwarka Sector 21', 'Nehru Stadium', 'Fun N Food WaterPark',
       'IGI Airport', 'Vasant Kunj', 'Prana

In [122]:
location_df.index = df.PropertyName
location_df.head()

,Bajghera Road,Palam Vihar Halt,DPSG Palam Vihar,Park Hospital,Gurgaon Railway Station,The NorthCap University,Dwarka Expy,Hyatt Place Gurgaon Udyog Vihar,"Dwarka Sector 21, Metro Station",Pacific D21 Mall,Indira Gandhi International Airport,Hamoni Golf Camp,Fun N Food Waterpark,Accenture DDC5,DPSG Palam Vihar Gurugram,"Park Hospital, Palam Vihar",Palam Vihar Halt Railway Station,Dwarka Sector 21 Metro Station,Dwarka Expressway,Fun N Food Water Park,Tau DeviLal Sports Complex,Hyatt Place,Altrade Business Centre,AIPL Business Club Sector 62,Heritage Xperiential Learning School,CK Birla Hospital,Paras Trinity Mall Sector 63,Rapid Metro Station Sector 56,De Adventure Park,Golf Course Ext Rd,DoubleTree by Hilton Hotel Gurgaon,KIIT College of Engineering Sohna Road,Mehrauli-Gurgaon Road,Nirvana Rd,TERI Golf Course,The Shikshiyan School,WTC Plaza,Luxus Haritma Resort,BSF Golf Course,Rions Hospital,Gurgaon,Dwarka Sector 21,Nehru Stadium,Fun N Food WaterPark,IGI Airport,Vasant Kunj,Pranavananda Int. School,DLF Site central office,Holiday Inn Gurugram Sector 90,Krishna Hospital,Royal Institute Of Science,Sapphire 83 Mall,NH48,Garhi Harsaru Junction,Manesar Golf Course,AapnoGhar,Vega Schools NH-8,DLF Corporate Greens,Miracles Apollo Cradle Hospital,Hyatt Regency Gurugram,NH 48,Golden Greens Golf & Resorts Limited,Mount Olympus Junior School,Miracles Apollo Hospital,NH -8,"Savoy Suites, Manesar",Golden Greens Golf & Resorts,IMT Manesar,Amity University Gurugram,Golf Course Extension Road,"Dwarka Expy, Sector 109","Euro International School, Sector- 109",Jai Sai Ram Hospital,Aryan Hospital,Idea Cosmic Plaza,Indira Gandhi Intl Airport,Royal Institute Of Science & Management,Pataudi Road,Holiday Inn Sector 90,RPS International School,Aarvy Healthcare Hospital,Iris Broadway Mall,Imperia Mindspace,AIPL Business Tower,Heritage School,"Lotus Valley Intl School, Gurgaon",Gurugram University,Sector 55-56 Metro Station,Omaxe Gurgaon Mall,Sushant University,"Badshahpur Sohna Rd Hwy,Sector 48",Naurangpur Cricket Stadium,Naurangpur Road,National Highway 48,Vatika Town Square-INXT,Ompee International School,Manesar Bus Stand,Yashlok Medical Centre,Euro International School,WorldMark Gurgaon,Capital Cyberscape,The Shriram Millennium School,DoubleTree by Hilton Hotel,Badshahpur Sohna Hwy,Nakhrola Stadium,Delhi - Jaipur Expressway,Vatika Town Square-INXT Mall,Savoy Suites,Bal Bharati Public School,Vatika Business Centre,Indira Gandhi Int. Airport,St. Xavier's High School,Miracles Apollo Cradle,Ambience Mall New,NH8,Hyatt Regency Gurgaon,Delhi Public School,Elan Miracle Mall,Miracles Apollo Cradle Spectra Hospital,Agri Business Management Collage,Delhi Jaipur Expressway,Grand Hyatt Gurgaon,Duke Horse Riding Club,PVR Drive In Cinema,W Pratiksha Hospital,Metro World Mall,Unicosmos School,Faridabad Gurgaon Road,Sohna Road,Bestech Business Tower,Appu Ghar,SkyJumper Trampoline Park,Axis Bank,KMP Expressway,Karma Lakelands,Jungle Safari & Trails,DPS Manesar,Medanta Hospital,Faridabad - Gurgaon Road,Lingaya's Lalita Devi Institute,ASF Insignia SEZ,Banjara Market Gurugram,Central Plaza Mall,"Paras Hospitals, Gurgaon",Badshahpur Sohna Rd Hwy,Vega School,Indian School of Hospitality,Vatika City Centre,Aatish Hospital,Info Technology Park Phase 2,Huda Metro Station,Southern Peripheral Rd,Global Ways School,Radisson Hotel,NH 248A,Sector 55/56 Metro Station,Mavens Inn,Sanar International Hospital,Sector 53-54 Metro Station,"IILM University, Gurugram",The Banyan Tree World School,The Big Tree Cafe,DLF Golf and Country Club,"Delhi Public School, Sector 84",Aarvy Hospital,DPG Degree College,Shivani public school,Baghera University,Kutumbh Hospital,Bijwasan Railway Station,Global Foyer Mall,Phase 2 Metro Station,Gurgaon Dreamz Mall,"Metro Hospital, Palam Vihar",Delhi Ajmer Expressway,Infinity Business Park,Huda metro station,Rion's Hospital,"Euro International School, Sector- 109.",Golf Course Ext Road,"Heritage Xperiential Learning, CRPF Rd",Sector 54 Chowk Metro Station

In [123]:
# fill na with max value assuming it should be far from this property
location_df.fillna(54000,inplace=True)
location_df.head()

,Bajghera Road,Palam Vihar Halt,DPSG Palam Vihar,Park Hospital,Gurgaon Railway Station,The NorthCap University,Dwarka Expy,Hyatt Place Gurgaon Udyog Vihar,"Dwarka Sector 21, Metro Station",Pacific D21 Mall,Indira Gandhi International Airport,Hamoni Golf Camp,Fun N Food Waterpark,Accenture DDC5,DPSG Palam Vihar Gurugram,"Park Hospital, Palam Vihar",Palam Vihar Halt Railway Station,Dwarka Sector 21 Metro Station,Dwarka Expressway,Fun N Food Water Park,Tau DeviLal Sports Complex,Hyatt Place,Altrade Business Centre,AIPL Business Club Sector 62,Heritage Xperiential Learning School,CK Birla Hospital,Paras Trinity Mall Sector 63,Rapid Metro Station Sector 56,De Adventure Park,Golf Course Ext Rd,DoubleTree by Hilton Hotel Gurgaon,KIIT College of Engineering Sohna Road,Mehrauli-Gurgaon Road,Nirvana Rd,TERI Golf Course,The Shikshiyan School,WTC Plaza,Luxus Haritma Resort,BSF Golf Course,Rions Hospital,Gurgaon,Dwarka Sector 21,Nehru Stadium,Fun N Food WaterPark,IGI Airport,Vasant Kunj,Pranavananda Int. School,DLF Site central office,Holiday Inn Gurugram Sector 90,Krishna Hospital,Royal Institute Of Science,Sapphire 83 Mall,NH48,Garhi Harsaru Junction,Manesar Golf Course,AapnoGhar,Vega Schools NH-8,DLF Corporate Greens,Miracles Apollo Cradle Hospital,Hyatt Regency Gurugram,NH 48,Golden Greens Golf & Resorts Limited,Mount Olympus Junior School,Miracles Apollo Hospital,NH -8,"Savoy Suites, Manesar",Golden Greens Golf & Resorts,IMT Manesar,Amity University Gurugram,Golf Course Extension Road,"Dwarka Expy, Sector 109","Euro International School, Sector- 109",Jai Sai Ram Hospital,Aryan Hospital,Idea Cosmic Plaza,Indira Gandhi Intl Airport,Royal Institute Of Science & Management,Pataudi Road,Holiday Inn Sector 90,RPS International School,Aarvy Healthcare Hospital,Iris Broadway Mall,Imperia Mindspace,AIPL Business Tower,Heritage School,"Lotus Valley Intl School, Gurgaon",Gurugram University,Sector 55-56 Metro Station,Omaxe Gurgaon Mall,Sushant University,"Badshahpur Sohna Rd Hwy,Sector 48",Naurangpur Cricket Stadium,Naurangpur Road,National Highway 48,Vatika Town Square-INXT,Ompee International School,Manesar Bus Stand,Yashlok Medical Centre,Euro International School,WorldMark Gurgaon,Capital Cyberscape,The Shriram Millennium School,DoubleTree by Hilton Hotel,Badshahpur Sohna Hwy,Nakhrola Stadium,Delhi - Jaipur Expressway,Vatika Town Square-INXT Mall,Savoy Suites,Bal Bharati Public School,Vatika Business Centre,Indira Gandhi Int. Airport,St. Xavier's High School,Miracles Apollo Cradle,Ambience Mall New,NH8,Hyatt Regency Gurgaon,Delhi Public School,Elan Miracle Mall,Miracles Apollo Cradle Spectra Hospital,Agri Business Management Collage,Delhi Jaipur Expressway,Grand Hyatt Gurgaon,Duke Horse Riding Club,PVR Drive In Cinema,W Pratiksha Hospital,Metro World Mall,Unicosmos School,Faridabad Gurgaon Road,Sohna Road,Bestech Business Tower,Appu Ghar,SkyJumper Trampoline Park,Axis Bank,KMP Expressway,Karma Lakelands,Jungle Safari & Trails,DPS Manesar,Medanta Hospital,Faridabad - Gurgaon Road,Lingaya's Lalita Devi Institute,ASF Insignia SEZ,Banjara Market Gurugram,Central Plaza Mall,"Paras Hospitals, Gurgaon",Badshahpur Sohna Rd Hwy,Vega School,Indian School of Hospitality,Vatika City Centre,Aatish Hospital,Info Technology Park Phase 2,Huda Metro Station,Southern Peripheral Rd,Global Ways School,Radisson Hotel,NH 248A,Sector 55/56 Metro Station,Mavens Inn,Sanar International Hospital,Sector 53-54 Metro Station,"IILM University, Gurugram",The Banyan Tree World School,The Big Tree Cafe,DLF Golf and Country Club,"Delhi Public School, Sector 84",Aarvy Hospital,DPG Degree College,Shivani public school,Baghera University,Kutumbh Hospital,Bijwasan Railway Station,Global Foyer Mall,Phase 2 Metro Station,Gurgaon Dreamz Mall,"Metro Hospital, Palam Vihar",Delhi Ajmer Expressway,Infinity Business Park,Huda metro station,Rion's Hospital,"Euro International School, Sector- 109.",Golf Course Ext Road,"Heritage Xperiential Learning, CRPF Rd",Sector 54 Chowk Metro Station

In [124]:
from sklearn.preprocessing import StandardScaler
# Initialize the scaler
scaler = StandardScaler()

# Apply the scaler to the entire dataframe
location_df_normalized = pd.DataFrame(scaler.fit_transform(location_df), columns=location_df.columns, index=location_df.index)
location_df_normalized.head()

,Bajghera Road,Palam Vihar Halt,DPSG Palam Vihar,Park Hospital,Gurgaon Railway Station,The NorthCap University,Dwarka Expy,Hyatt Place Gurgaon Udyog Vihar,"Dwarka Sector 21, Metro Station",Pacific D21 Mall,Indira Gandhi International Airport,Hamoni Golf Camp,Fun N Food Waterpark,Accenture DDC5,DPSG Palam Vihar Gurugram,"Park Hospital, Palam Vihar",Palam Vihar Halt Railway Station,Dwarka Sector 21 Metro Station,Dwarka Expressway,Fun N Food Water Park,Tau DeviLal Sports Complex,Hyatt Place,Altrade Business Centre,AIPL Business Club Sector 62,Heritage Xperiential Learning School,CK Birla Hospital,Paras Trinity Mall Sector 63,Rapid Metro Station Sector 56,De Adventure Park,Golf Course Ext Rd,DoubleTree by Hilton Hotel Gurgaon,KIIT College of Engineering Sohna Road,Mehrauli-Gurgaon Road,Nirvana Rd,TERI Golf Course,The Shikshiyan School,WTC Plaza,Luxus Haritma Resort,BSF Golf Course,Rions Hospital,Gurgaon,Dwarka Sector 21,Nehru Stadium,Fun N Food WaterPark,IGI Airport,Vasant Kunj,Pranavananda Int. School,DLF Site central office,Holiday Inn Gurugram Sector 90,Krishna Hospital,Royal Institute Of Science,Sapphire 83 Mall,NH48,Garhi Harsaru Junction,Manesar Golf Course,AapnoGhar,Vega Schools NH-8,DLF Corporate Greens,Miracles Apollo Cradle Hospital,Hyatt Regency Gurugram,NH 48,Golden Greens Golf & Resorts Limited,Mount Olympus Junior School,Miracles Apollo Hospital,NH -8,"Savoy Suites, Manesar",Golden Greens Golf & Resorts,IMT Manesar,Amity University Gurugram,Golf Course Extension Road,"Dwarka Expy, Sector 109","Euro International School, Sector- 109",Jai Sai Ram Hospital,Aryan Hospital,Idea Cosmic Plaza,Indira Gandhi Intl Airport,Royal Institute Of Science & Management,Pataudi Road,Holiday Inn Sector 90,RPS International School,Aarvy Healthcare Hospital,Iris Broadway Mall,Imperia Mindspace,AIPL Business Tower,Heritage School,"Lotus Valley Intl School, Gurgaon",Gurugram University,Sector 55-56 Metro Station,Omaxe Gurgaon Mall,Sushant University,"Badshahpur Sohna Rd Hwy,Sector 48",Naurangpur Cricket Stadium,Naurangpur Road,National Highway 48,Vatika Town Square-INXT,Ompee International School,Manesar Bus Stand,Yashlok Medical Centre,Euro International School,WorldMark Gurgaon,Capital Cyberscape,The Shriram Millennium School,DoubleTree by Hilton Hotel,Badshahpur Sohna Hwy,Nakhrola Stadium,Delhi - Jaipur Expressway,Vatika Town Square-INXT Mall,Savoy Suites,Bal Bharati Public School,Vatika Business Centre,Indira Gandhi Int. Airport,St. Xavier's High School,Miracles Apollo Cradle,Ambience Mall New,NH8,Hyatt Regency Gurgaon,Delhi Public School,Elan Miracle Mall,Miracles Apollo Cradle Spectra Hospital,Agri Business Management Collage,Delhi Jaipur Expressway,Grand Hyatt Gurgaon,Duke Horse Riding Club,PVR Drive In Cinema,W Pratiksha Hospital,Metro World Mall,Unicosmos School,Faridabad Gurgaon Road,Sohna Road,Bestech Business Tower,Appu Ghar,SkyJumper Trampoline Park,Axis Bank,KMP Expressway,Karma Lakelands,Jungle Safari & Trails,DPS Manesar,Medanta Hospital,Faridabad - Gurgaon Road,Lingaya's Lalita Devi Institute,ASF Insignia SEZ,Banjara Market Gurugram,Central Plaza Mall,"Paras Hospitals, Gurgaon",Badshahpur Sohna Rd Hwy,Vega School,Indian School of Hospitality,Vatika City Centre,Aatish Hospital,Info Technology Park Phase 2,Huda Metro Station,Southern Peripheral Rd,Global Ways School,Radisson Hotel,NH 248A,Sector 55/56 Metro Station,Mavens Inn,Sanar International Hospital,Sector 53-54 Metro Station,"IILM University, Gurugram",The Banyan Tree World School,The Big Tree Cafe,DLF Golf and Country Club,"Delhi Public School, Sector 84",Aarvy Hospital,DPG Degree College,Shivani public school,Baghera University,Kutumbh Hospital,Bijwasan Railway Station,Global Foyer Mall,Phase 2 Metro Station,Gurgaon Dreamz Mall,"Metro Hospital, Palam Vihar",Delhi Ajmer Expressway,Infinity Business Park,Huda metro station,Rion's Hospital,"Euro International School, Sector- 109.",Golf Course Ext Road,"Heritage Xperiential Learning, CRPF Rd",Sector 54 Chowk Metro Station

In [125]:
cosine_sim3 = cosine_similarity(location_df_normalized)
cosine_sim3.shape

(246, 246)

In [126]:
def recommend_properties_with_scores(property_name, top_n=247):
    
    cosine_sim_matrix = 0.3*cosine_sim1 + 0.2*cosine_sim2 + 0.5*cosine_sim3
    # cosine_sim_matrix = cosine_sim3
    
    # Get the similarity scores for the property using its name as the index
    sim_scores = list(enumerate(cosine_sim_matrix[location_df_normalized.index.get_loc(property_name)]))
    
    # Sort properties based on the similarity scores
    sorted_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # Get the indices and scores of the top_n most similar properties
    top_indices = [i[0] for i in sorted_scores[1:top_n+1]]
    top_scores = [i[1] for i in sorted_scores[1:top_n+1]]
    
    # Retrieve the names of the top properties using the indices
    top_properties = location_df_normalized.index[top_indices].tolist()
    
    # Create a dataframe with the results
    recommendations_df = pd.DataFrame({
        'PropertyName': top_properties,
        'SimilarityScore': top_scores
    })
    
    return recommendations_df

# Test the recommender function using a property name
recommend_properties_with_scores('Ireo Victory Valley').head(5)

,PropertyName,SimilarityScore
0,Pioneer Urban Presidia,0.277362
1,Ambience Creacions,0.271830
2,Bestech Park View Grand Spa,0.270368
3,DLF The Crest,0.228960
4,Pioneer Araya,0.224057


In [127]:
(3*cosine_sim3 + 5*cosine_sim2 + 6*cosine_sim1).shape

(246, 246)

In [128]:
recommend_properties_with_scores('DLF The Camellias').head(5)

,PropertyName,SimilarityScore
0,Salcon The Verandas,0.250802
1,DLF The Magnolias,0.203515
2,DLF The Aralias,0.168890
3,Parsvnath Exotica,0.115149
4,Anant Raj Estates,0.111189


In [129]:
location_df.head()

,Bajghera Road,Palam Vihar Halt,DPSG Palam Vihar,Park Hospital,Gurgaon Railway Station,The NorthCap University,Dwarka Expy,Hyatt Place Gurgaon Udyog Vihar,"Dwarka Sector 21, Metro Station",Pacific D21 Mall,Indira Gandhi International Airport,Hamoni Golf Camp,Fun N Food Waterpark,Accenture DDC5,DPSG Palam Vihar Gurugram,"Park Hospital, Palam Vihar",Palam Vihar Halt Railway Station,Dwarka Sector 21 Metro Station,Dwarka Expressway,Fun N Food Water Park,Tau DeviLal Sports Complex,Hyatt Place,Altrade Business Centre,AIPL Business Club Sector 62,Heritage Xperiential Learning School,CK Birla Hospital,Paras Trinity Mall Sector 63,Rapid Metro Station Sector 56,De Adventure Park,Golf Course Ext Rd,DoubleTree by Hilton Hotel Gurgaon,KIIT College of Engineering Sohna Road,Mehrauli-Gurgaon Road,Nirvana Rd,TERI Golf Course,The Shikshiyan School,WTC Plaza,Luxus Haritma Resort,BSF Golf Course,Rions Hospital,Gurgaon,Dwarka Sector 21,Nehru Stadium,Fun N Food WaterPark,IGI Airport,Vasant Kunj,Pranavananda Int. School,DLF Site central office,Holiday Inn Gurugram Sector 90,Krishna Hospital,Royal Institute Of Science,Sapphire 83 Mall,NH48,Garhi Harsaru Junction,Manesar Golf Course,AapnoGhar,Vega Schools NH-8,DLF Corporate Greens,Miracles Apollo Cradle Hospital,Hyatt Regency Gurugram,NH 48,Golden Greens Golf & Resorts Limited,Mount Olympus Junior School,Miracles Apollo Hospital,NH -8,"Savoy Suites, Manesar",Golden Greens Golf & Resorts,IMT Manesar,Amity University Gurugram,Golf Course Extension Road,"Dwarka Expy, Sector 109","Euro International School, Sector- 109",Jai Sai Ram Hospital,Aryan Hospital,Idea Cosmic Plaza,Indira Gandhi Intl Airport,Royal Institute Of Science & Management,Pataudi Road,Holiday Inn Sector 90,RPS International School,Aarvy Healthcare Hospital,Iris Broadway Mall,Imperia Mindspace,AIPL Business Tower,Heritage School,"Lotus Valley Intl School, Gurgaon",Gurugram University,Sector 55-56 Metro Station,Omaxe Gurgaon Mall,Sushant University,"Badshahpur Sohna Rd Hwy,Sector 48",Naurangpur Cricket Stadium,Naurangpur Road,National Highway 48,Vatika Town Square-INXT,Ompee International School,Manesar Bus Stand,Yashlok Medical Centre,Euro International School,WorldMark Gurgaon,Capital Cyberscape,The Shriram Millennium School,DoubleTree by Hilton Hotel,Badshahpur Sohna Hwy,Nakhrola Stadium,Delhi - Jaipur Expressway,Vatika Town Square-INXT Mall,Savoy Suites,Bal Bharati Public School,Vatika Business Centre,Indira Gandhi Int. Airport,St. Xavier's High School,Miracles Apollo Cradle,Ambience Mall New,NH8,Hyatt Regency Gurgaon,Delhi Public School,Elan Miracle Mall,Miracles Apollo Cradle Spectra Hospital,Agri Business Management Collage,Delhi Jaipur Expressway,Grand Hyatt Gurgaon,Duke Horse Riding Club,PVR Drive In Cinema,W Pratiksha Hospital,Metro World Mall,Unicosmos School,Faridabad Gurgaon Road,Sohna Road,Bestech Business Tower,Appu Ghar,SkyJumper Trampoline Park,Axis Bank,KMP Expressway,Karma Lakelands,Jungle Safari & Trails,DPS Manesar,Medanta Hospital,Faridabad - Gurgaon Road,Lingaya's Lalita Devi Institute,ASF Insignia SEZ,Banjara Market Gurugram,Central Plaza Mall,"Paras Hospitals, Gurgaon",Badshahpur Sohna Rd Hwy,Vega School,Indian School of Hospitality,Vatika City Centre,Aatish Hospital,Info Technology Park Phase 2,Huda Metro Station,Southern Peripheral Rd,Global Ways School,Radisson Hotel,NH 248A,Sector 55/56 Metro Station,Mavens Inn,Sanar International Hospital,Sector 53-54 Metro Station,"IILM University, Gurugram",The Banyan Tree World School,The Big Tree Cafe,DLF Golf and Country Club,"Delhi Public School, Sector 84",Aarvy Hospital,DPG Degree College,Shivani public school,Baghera University,Kutumbh Hospital,Bijwasan Railway Station,Global Foyer Mall,Phase 2 Metro Station,Gurgaon Dreamz Mall,"Metro Hospital, Palam Vihar",Delhi Ajmer Expressway,Infinity Business Park,Huda metro station,Rion's Hospital,"Euro International School, Sector- 109.",Golf Course Ext Road,"Heritage Xperiential Learning, CRPF Rd",Sector 54 Chowk Metro Station

In [130]:
# save pickle file
import pickle

with open('./pickle/location_distance.pkl', 'wb') as file:
    pickle.dump(location_df, file)

In [131]:
with open('./pickle/cosine_sim1.pkl', 'wb') as file:
    pickle.dump(cosine_sim1, file)

with open('./pickle/cosine_sim2.pkl', 'wb') as file:
    pickle.dump(cosine_sim2, file)

with open('./pickle/cosine_sim3.pkl', 'wb') as file:
    pickle.dump(cosine_sim3, file)